In [ ]:
import re
import pandas as pd

In [ ]:
f= open('temp-chat.txt','r',encoding='utf-8')

In [ ]:
data = f.read()

In [ ]:
print(type(data))

<class 'str'>


In [ ]:
pattern  = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-332419532.py:1: SyntaxWarning: invalid escape sequence '\d'
  pattern  = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'


In [ ]:
messages = re.split(pattern,data)[1:]
len(messages)

290

In [ ]:
dates = re.findall(pattern,data)
len(dates)

290

In [ ]:
df = pd.DataFrame({'user_message':messages,'message_date':dates})
#convert message data type
df['message_date'] = pd.to_datetime(df['message_date'],format='%d/%m/%y, %H:%M - ')
df.rename(columns={'message_date':'date'},inplace=True)
df.head()

In [ ]:
df.shape

(290, 2)

In [ ]:
#Seperate users and messages
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s',message)
    if entry[1:]:
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])
df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'],inplace=True)
df.head()


In [ ]:
df['year'] = df['date'].dt.year

In [ ]:
df['month'] = df['date'].dt.month_name()

In [ ]:
df['day'] = df['date'].dt.day

In [ ]:
df['hour'] = df['date'].dt.hour

In [ ]:
df['minute'] = df['date'].dt.minute

In [ ]:
words=[]
for message in df['message']:
    words.extend(message.split())

In [ ]:
len(words)

1313

In [ ]:
!pip install urlextract


In [ ]:
from urlextract import URLExtract
extractor = URLExtract()

In [ ]:
links=[]
for message in df['message']:
  links.extend(extractor.find_urls(message))

In [ ]:
len(links)

9

In [ ]:
x=df['user'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
name=x.index
count=x.values

In [ ]:
plt.bar(name,count)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
round(df['user'].value_counts().head()/df.shape[0]*100,2).reset_index().rename(columns={'index':'name','user':'percent'})

In [ ]:
temp = df[df['user'] != 'group_notification']
temp=temp[temp['message'] != '<Media omitted>\n']

In [ ]:
f= open('stop_hinglish.txt','r')
stop_words = f.read()

In [ ]:
words=[]
for message in temp['message']:
  for word in message.lower().split():
    if word not in stop_words:
      words.append(word)

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 10.4 MB/s eta 0:00:00


In [ ]:
import emoji

emojis = []
for message in df['message']:
    emojis.extend([c for c in message if c in emoji.EMOJI_DATA])


In [ ]:
pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))))

In [ ]:
df['month_num'] = df['date'].dt.month

In [ ]:
timeline=df.groupby(['year','month_num','month']).count()['message'].reset_index()

In [ ]:
time=[]
for i in range(timeline.shape[0]):
  time.append(timeline['month'][i] + "-" + str(timeline['year'][i]))

In [ ]:
timeline['time']=time

In [ ]:
plt.plot(timeline['time'],timeline['message'])
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
df['only_date']=df['date'].dt.date

In [ ]:
daily_timeline=df.groupby(['only_date']).count()['message'].reset_index()

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(daily_timeline['only_date'],daily_timeline['message'])
plt.show()

In [ ]:
df['day_name']=df['date'].dt.day_name()

In [ ]:
df['day_name'].value_counts()

In [ ]:
df['month'].value_counts()

In [ ]:
period=[]
for hour in df[['day_name','hour']]['hour']:
  if hour==23:
    period.append(str(hour) + "-" + str('00'))
  elif hour==0:
    period.append(str('00') + "-" + str(hour+1))
  else:
    period.append(str(hour) + "-" + str(hour+1))

In [ ]:
df['period']=period

In [ ]:
df.sample(5)

In [ ]:
import seaborn as sns
plt.figure(figsize=(20,6))
sns.heatmap(df.pivot_table(index='day_name',columns='period',values='message',aggfunc='count').fillna(0))
plt.yticks(rotation='horizontal')
plt.show()